<a href="https://colab.research.google.com/github/vishnuvardhan2520/personalized-yoga-recommend-stsyem-/blob/main/yoga%20recommend%20system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

np.random.seed(42)

# Generate data
n_samples = 10000
ages = np.random.randint(18, 65, n_samples)
genders = np.random.choice(['Male', 'Female'], n_samples)
bmi = np.round(np.random.uniform(18, 35, n_samples), 1)
experience = np.random.choice(['Newbie', 'Intermediate', 'Advanced'], n_samples)
goal = np.random.choice(['Weight Loss', 'Strength', 'Flexibility', 'Stress Relief'], n_samples)
preferred_time = np.random.choice(['Morning', 'Evening', 'Night'], n_samples)

# Recommendation logic
def recommend_plan(exp, g):
    if exp == 'Newbie':
        if g in ['Weight Loss', 'Stress Relief']:
            return 'Beginner Yoga'
        elif g == 'Flexibility':
            return 'Stretching Exercises'
        else:
            return 'Cardio Blast'
    elif exp == 'Intermediate':
        if g == 'Weight Loss':
            return 'Power Yoga'
        elif g == 'Strength':
            return 'Strength Training'
        else:
            return 'Meditation'
    else:  # Advanced
        if g == 'Strength':
            return 'Strength Training'
        elif g == 'Flexibility':
            return 'Power Yoga'
        else:
            return 'Meditation'

recommended_plan = [recommend_plan(e, g) for e, g in zip(experience, goal)]

# Create DataFrame
df = pd.DataFrame({
    'Age': ages,
    'Gender': genders,
    'BMI': bmi,
    'Experience_Level': experience,
    'Fitness_Goal': goal,
    'Preferred_Time': preferred_time,
    'Recommended_Plan': recommended_plan
})

# Create separate label encoders for each categorical column
encoders = {}
for col in ['Gender', 'Experience_Level', 'Fitness_Goal', 'Preferred_Time', 'Recommended_Plan']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le

# Prepare features and target
X = df.drop('Recommended_Plan', axis=1)
y = df['Recommended_Plan']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest
rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=15,
    min_samples_split=5,
    random_state=42
)
rf_model.fit(X_train, y_train)

# Evaluation
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("\nRandom Forest Accuracy:", accuracy * 100, "%")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Predict for a new user
new_user = pd.DataFrame({
    'Age': [30],
    'Gender': encoders['Gender'].transform(['Female']),
    'BMI': [24.5],
    'Experience_Level': encoders['Experience_Level'].transform(['Intermediate']),
    'Fitness_Goal': encoders['Fitness_Goal'].transform(['Strength']),
    'Preferred_Time': encoders['Preferred_Time'].transform(['Evening'])
})

recommendation = rf_model.predict(new_user)
print("\nRecommended Plan for New User:", encoders['Recommended_Plan'].inverse_transform(recommendation))





Random Forest Accuracy: 100.0 %

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       338
           1       1.00      1.00      1.00       172
           2       1.00      1.00      1.00       675
           3       1.00      1.00      1.00       325
           4       1.00      1.00      1.00       345
           5       1.00      1.00      1.00       145

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000


Recommended Plan for New User: ['Strength Training']
